In [70]:
!pip install datasets

In [71]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("GonzaloA/fake_news")

# Display the dataset structure
print(ds)

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 24353
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
})


In [72]:
# View the first example in the dataset
print(ds["train"][0])

# View the first five examples
print(ds["train"][:5])

{'Unnamed: 0': 0, 'title': ' ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)', 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the caption

In [73]:
print(ds["train"][:5])

{'Unnamed: 0': [0, 1, 2, 3, 4], 'title': [' ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)', ' Trump’s Favorite News Channel Tries To Soothe His Battered Ego – Gets Taken To The Cleaners', 'Russia warns Iraq, Kurds not to destabilize Middle East after Kurdish vote', 'WATCH STEVE SCALISE Throw A Strike At The Nationals Baseball Game [Video]', ' Trump Will HATE What Stephen Colbert Just Did To Him – It’s Pure Comedy Genius (VIDEO)'], 'text': ['Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes rega

In [74]:
# Check for missing values
print(ds["train"].filter(lambda x: x["text"] is None))

Dataset({
    features: ['Unnamed: 0', 'title', 'text', 'label'],
    num_rows: 0
})


In [75]:
ds = ds.filter(lambda x: x["text"] is not None)

In [76]:
def to_lowercase(example):
    example["text"] = example["text"].lower()
    return example

ds = ds.map(to_lowercase)

In [77]:
import re

def remove_special_characters(example):
    example["text"] = re.sub(r"[^a-zA-Z0-9\s]", "", example["text"])
    return example

ds = ds.map(remove_special_characters)

In [78]:
!pip install transformers

In [79]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

ds = ds.map(tokenize_data, batched=True)

In [81]:
ds = ds["train"].train_test_split(test_size=0.2)

In [82]:
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Load dataset and split
ds = load_dataset("GonzaloA/fake_news")
ds = ds["train"].train_test_split(test_size=0.2)

# Vectorize text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform([x["text"] for x in ds["train"]])
X_test = vectorizer.transform([x["text"] for x in ds["test"]])

# Extract labels
y_train = [x["label"] for x in ds["train"]]
y_test = [x["label"] for x in ds["test"]]

# Train logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Repo card metadata block was not found. Setting CardData to empty.


Accuracy: 0.9698213919113119
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2261
           1       0.97      0.97      0.97      2610

    accuracy                           0.97      4871
   macro avg       0.97      0.97      0.97      4871
weighted avg       0.97      0.97      0.97      4871



In [83]:
from datasets import load_dataset
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Load dataset and split
ds = load_dataset("GonzaloA/fake_news")
ds = ds["train"].train_test_split(test_size=0.2)

# Vectorize text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform([x["text"] for x in ds["train"]])
X_test = vectorizer.transform([x["text"] for x in ds["test"]])

# Extract labels
y_train = [x["label"] for x in ds["train"]]
y_test = [x["label"] for x in ds["test"]]

# Initialize and train the Gradient Boosting model
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = gb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))


Repo card metadata block was not found. Setting CardData to empty.


Accuracy: 0.9802915212482036
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2310
           1       0.99      0.97      0.98      2561

    accuracy                           0.98      4871
   macro avg       0.98      0.98      0.98      4871
weighted avg       0.98      0.98      0.98      4871



In [84]:
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Load dataset and split
ds = load_dataset("GonzaloA/fake_news")
ds = ds["train"].train_test_split(test_size=0.2)

# Vectorize text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform([x["text"] for x in ds["train"]])
X_test = vectorizer.transform([x["text"] for x in ds["test"]])

# Extract labels
y_train = [x["label"] for x in ds["train"]]
y_test = [x["label"] for x in ds["test"]]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))


Repo card metadata block was not found. Setting CardData to empty.


Accuracy: 0.9784438513652227
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2221
           1       0.99      0.97      0.98      2650

    accuracy                           0.98      4871
   macro avg       0.98      0.98      0.98      4871
weighted avg       0.98      0.98      0.98      4871



In [85]:
from datasets import load_dataset
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Load dataset and split
ds = load_dataset("GonzaloA/fake_news")
ds = ds["train"].train_test_split(test_size=0.2)

# Vectorize text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform([x["text"] for x in ds["train"]])
X_test = vectorizer.transform([x["text"] for x in ds["test"]])

# Extract labels
y_train = [x["label"] for x in ds["train"]]
y_test = [x["label"] for x in ds["test"]]

# Initialize and train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = nb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Repo card metadata block was not found. Setting CardData to empty.


Accuracy: 0.9189078218025046
              precision    recall  f1-score   support

           0       0.91      0.92      0.91      2253
           1       0.93      0.92      0.92      2618

    accuracy                           0.92      4871
   macro avg       0.92      0.92      0.92      4871
weighted avg       0.92      0.92      0.92      4871



In [86]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
ds = load_dataset("GonzaloA/fake_news")
ds = ds["train"].train_test_split(test_size=0.2)

# Vectorize text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform([x["text"] for x in ds["train"]])
X_test = vectorizer.transform([x["text"] for x in ds["test"]])

# Extract labels
y_train = [x["label"] for x in ds["train"]]
y_test = [x["label"] for x in ds["test"]]

# Initialize each model
log_reg_model = LogisticRegression(max_iter=1000)
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
nb_model = MultinomialNB()

# Train all models on the training data
log_reg_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
nb_model.fit(X_train, y_train)

# List of models for evaluation
models = {
    "Logistic Regression": log_reg_model,
    "Gradient Boosting": gb_model,
    "Random Forest": rf_model,
    "Naive Bayes": nb_model
}

# Evaluate each model
for model_name, model in models.items():
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Print model's name, accuracy, and classification report
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("="*50)




Repo card metadata block was not found. Setting CardData to empty.


Model: Logistic Regression
Accuracy: 0.9692
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      2258
           1       0.97      0.97      0.97      2613

    accuracy                           0.97      4871
   macro avg       0.97      0.97      0.97      4871
weighted avg       0.97      0.97      0.97      4871

Model: Gradient Boosting
Accuracy: 0.9801
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2258
           1       0.99      0.97      0.98      2613

    accuracy                           0.98      4871
   macro avg       0.98      0.98      0.98      4871
weighted avg       0.98      0.98      0.98      4871

Model: Random Forest
Accuracy: 0.9817
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2258
           1       0.99      0.97      0.98 

In [87]:
 new_texts = [
     "Why is Elon Musk becoming Donald Trump's efficiency tsar Billionaire Elon Musk has been tasked with leading incoming President Donald Trump's new Department of Government Efficiency (Doge) In a statement on social media, the US president-elect said Musk - along with former Republican presidential candidate Vivek Ramaswamy - would dismantle government bureaucracy, slash excess regulations, cut wasteful expenditures and restructure federal agencies It is a role that the tech entrepreneur has arguably prepared for through his business leadership and one he has spent months arguing for But it is also one that is expected to garner him influence over government policy and the regulatory environment his enterprises exist in"
  ]

In [88]:
new_texts_vectorized = vectorizer.transform(new_texts)

In [93]:
# Define a function to make predictions for the new texts
def check_fake_news(models, new_texts_vectorized):
    for model_name, model in models.items():
        y_pred = model.predict(new_texts_vectorized)
        
        # Print out the predictions
        print(f"Model: {model_name}")
        for i, text in enumerate(new_texts):
            print(f"Text {i+1}: {'Fake' if y_pred[i] == 0 else 'Real'}")
        print("=" * 50)

# Check predictions (make sure this line is at the correct indentation level)
check_fake_news(models, new_texts_vectorized)

Model: Logistic Regression
Text 1: Real
Model: Gradient Boosting
Text 1: Fake
Model: Random Forest
Text 1: Fake
Model: Naive Bayes
Text 1: Real


In [94]:
from newspaper import Article
from sklearn.feature_extraction.text import TfidfVectorizer

# Define function to extract article text from a URL
def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

# Example usage: Extract article text
url = 'https://www.livemint.com/money/personal-finance/income-tax-will-fm-nirmala-sitharaman-announce-tax-relief-for-taxpayers-earning-up-to-15-lakh-in-budget-2025-itr-filing-11735360922393.html'
article_text = extract_article(url)

# Print the extracted text
print("Extracted Article Text:\n", article_text)

# Assuming 'vectorizer' is already trained with your dataset
# Vectorize the new article text
new_text_vectorized = vectorizer.transform([article_text])

# Function to check if the article is fake or real
def check_fake_news(models, new_text_vectorized):
    for model_name, model in models.items():
        y_pred = model.predict(new_text_vectorized)
        
        # Print the prediction for each model
        print(f"Model: {model_name}")
        print(f"Prediction: {'Fake' if y_pred[0] == 0 else 'Real'}")
        print("=" * 50)

# Run the fake news check
check_fake_news(models, new_text_vectorized)


Extracted Article Text:
 Every year, India’s salaried class eagerly awaits relief in income tax. This year, the middle class is also penning all their hopes for FM Nirmala Sitharaman, when she presents Budget 2025 on February 1. According to a Reuters report, the government is actively considering tax relief for the middle class, particularly earning up to ₹15 lakh annually. The 2025 Budget, to be tabled on February 1, may include an announcement on the tax breaks. This action is intended to boost consumer spending and stimulate the economy. Livemint couldn't independently verify the information.

Income tax budget: Who will benefit? If this is implemented, millions of taxpayers might benefit greatly, especially city dwellers struggling with high living costs. According to the Reuters report, those choosing the 2020 tax scheme, which excludes exclusions like those for housing rents, would be eligible for this.

Also Read | How much cash can you receive in your bank savings account in a

In [95]:
import numpy as np
from newspaper import Article
from sklearn.feature_extraction.text import TfidfVectorizer

# Define function to extract article text from a URL
def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

# Example usage: Extract article text from the provided URL
url = 'https://www.livemint.com/money/personal-finance/income-tax-will-fm-nirmala-sitharaman-announce-tax-relief-for-taxpayers-earning-up-to-15-lakh-in-budget-2025-itr-filing-11735360922393.html'  # Example article URL
article_text = extract_article(url)

# Print the extracted text
print("Extracted Article Text:\n", article_text)

# Assuming 'vectorizer' is already trained with your dataset
# Vectorize the new article text using the fitted vectorizer
new_text_vectorized = vectorizer.transform([article_text])

# Define the model accuracies (these should match the models you're using)
model_accuracies = [0.9717, 0.9811, 0.9780, 0.9230]  # Logistic Regression, Gradient Boosting, Random Forest, Naive Bayes

# Function to check if the article is fake or real with weighted decision
def check_fake_news_weighted(models, new_text_vectorized, model_accuracies):
    model_predictions = []
    
    # Get predictions from each model
    for i, (model_name, model) in enumerate(models.items()):
        y_pred = model.predict(new_text_vectorized)
        model_predictions.append(y_pred[0])  # Store the prediction of the first (and only) text

    # Initialize weighted sum of predictions
    weighted_sum = 0
    
    # Calculate the weighted sum of the predictions based on accuracies
    for i, prediction in enumerate(model_predictions):
        weighted_sum += prediction * model_accuracies[i]  # Multiply each model's prediction by its accuracy (weight)

    # Final decision: sum of weighted predictions
    if weighted_sum > 0:
        final_prediction = 1  # Consider "Real" if weighted sum is positive
    else:
        final_prediction = 0  # Consider "Fake" if weighted sum is zero or negative
    
    return 'Real' if final_prediction == 1 else 'Fake'

# Example models (These need to be defined or loaded based on your previous code)
# Assuming you already have these models loaded and trained:
models = {
    "Logistic Regression": log_reg_model,  # Your trained Logistic Regression model
    "Gradient Boosting": gb_model,        # Your trained Gradient Boosting model
    "Random Forest": rf_model,            # Your trained Random Forest model
    "Naive Bayes": nb_model               # Your trained Naive Bayes model
}

# Run the fake news check with weighted decision
final_result = check_fake_news_weighted(models, new_text_vectorized, model_accuracies)
print(f"Final Prediction: {final_result}")


Extracted Article Text:
 Every year, India’s salaried class eagerly awaits relief in income tax. This year, the middle class is also penning all their hopes for FM Nirmala Sitharaman, when she presents Budget 2025 on February 1. According to a Reuters report, the government is actively considering tax relief for the middle class, particularly earning up to ₹15 lakh annually. The 2025 Budget, to be tabled on February 1, may include an announcement on the tax breaks. This action is intended to boost consumer spending and stimulate the economy. Livemint couldn't independently verify the information.

Income tax budget: Who will benefit? If this is implemented, millions of taxpayers might benefit greatly, especially city dwellers struggling with high living costs. According to the Reuters report, those choosing the 2020 tax scheme, which excludes exclusions like those for housing rents, would be eligible for this.

Also Read | How much cash can you receive in your bank savings account in a

In [67]:
import numpy as np
from newspaper import Article
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils.validation import check_is_fitted

# Define function to extract article text from a URL
def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

# Example usage: Extract article text from the provided URL
url = 'https://www.bbc.com/news/articles/c6230646435o'  # Example article URL
article_text = extract_article(url)

# Print the extracted text
print("Extracted Article Text:\n", article_text)

# Training vectorizer (ensure you use your dataset for fitting)
# Assuming 'train_data' is a dictionary containing the keys 'text' and 'label'
train_data = {
    'text': ["sample text 1", "sample text 2", "sample text 3", "sample text 4"],  # Replace with your actual training data
    'label': [1, 0, 1, 0]  # Replace with your actual training labels
}

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['text'])
y_train = np.array(train_data['label'])

# Train models
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Verify all models are fitted
models = {
    "Logistic Regression": log_reg_model,
    "Gradient Boosting": gb_model,
    "Random Forest": rf_model,
    "Naive Bayes": nb_model
}

for model_name, model in models.items():
    try:
        check_is_fitted(model)
    except NotFittedError:
        raise ValueError(f"{model_name} is not fitted. Train it before use.")

# Vectorize the new article text using the fitted vectorizer
new_text_vectorized = vectorizer.transform([article_text])

# Define the model accuracies (these should match the models you're using)
model_accuracies = [0.9717, 0.9811, 0.9780, 0.9230]  # Logistic Regression, Gradient Boosting, Random Forest, Naive Bayes

# Function to check if the article is fake or real with weighted decision
def check_fake_news_weighted(models, new_text_vectorized, model_accuracies):
    model_predictions = []
    
    # Get predictions from each model
    for i, (model_name, model) in enumerate(models.items()):
        y_pred = model.predict(new_text_vectorized)
        model_predictions.append(y_pred[0])  # Store the prediction of the first (and only) text

    # Initialize weighted sum of predictions
    weighted_sum = 0
    
    # Calculate the weighted sum of the predictions based on accuracies
    for i, prediction in enumerate(model_predictions):
        weighted_sum += prediction * model_accuracies[i]  # Multiply each model's prediction by its accuracy (weight)

    # Final decision: sum of weighted predictions
    if weighted_sum > 0:
        final_prediction = 1  # Consider "Real" if weighted sum is positive
    else:
        final_prediction = 0  # Consider "Fake" if weighted sum is zero or negative
    
    return 'Real' if final_prediction == 1 else 'Fake'

# Run the fake news check with weighted decision
final_result = check_fake_news_weighted(models, new_text_vectorized, model_accuracies)
print(f"Final Prediction: {final_result}")



Extracted Article Text:
 Twenty years on: 'My boat was metres from the shore when the tsunami hit'

BBC Just as we pulled out from the harbour, our boat lurched and suddenly the jetty next to where we had boarded collapsed into the sea

Boxing Day, 2004. When the earthquake struck at 06:30 (01:00 GMT), I was on a ferry, headed towards Havelock – an island in the Indian archipelago of Andaman and Nicobar. Known for its silver sand and clear blue waters, the Radhanagar beach there had recently been crowned "Asia's Best Beach" by Time magazine. My best friend from college and her family had lived in Port Blair, the capital of the archipelago, for a decade and a half, but this was my first visit to the islands, where I had arrived on Christmas Eve. We had planned to spend three days in Havelock and in the morning we packed snacks and sandwiches, gathered excited children and headed out to catch the ferry from Phoenix Bay jetty in Port Blair. Not wanting to miss out on anything, I was stand

In [96]:
import pickle

In [97]:
import joblib

# Save vectorizer and model
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(model, "log_reg_model.pkl")
joblib.dump(model, "rf_model.pkl")
joblib.dump(model, "gb_model.pkl")
joblib.dump(model, "nb_model.pkl")
print("Vectorizer and model saved successfully.")


Vectorizer and model saved successfully.


In [63]:
# Verify compatibility between vectorizer and model
vectorizer_vocab_size = len(vectorizer.get_feature_names_out())

# Check model type and retrieve input size
if isinstance(log_reg_model, LogisticRegression):
    model_input_size = log_reg_model.coef_.shape[1]
elif isinstance(log_reg_model, MultinomialNB):
    model_input_size = log_reg_model.feature_log_prob_.shape[1]
else:
    raise TypeError(f"Unsupported model type: {type(log_reg_model)}")

# Ensure compatibility
if vectorizer_vocab_size != model_input_size:
    raise ValueError(
        f"Vectorizer vocabulary size ({vectorizer_vocab_size}) does not match model input size ({model_input_size}). "
        "Ensure the vectorizer and models are from the same training pipeline."
    )


ValueError: Vectorizer vocabulary size (5) does not match model input size (2). Ensure the vectorizer and models are from the same training pipeline.

In [61]:
# Load the vectorizer and model
vectorizer = joblib.load("tfidf_vectorizer.pkl")
log_reg_model = joblib.load("log_reg_model.pkl")

# Verify compatibility
vectorizer_vocab_size = len(vectorizer.get_feature_names_out())
log_reg_input_size = log_reg_model.coef_.shape[1]

if vectorizer_vocab_size != log_reg_input_size:
    raise ValueError(
        f"Vectorizer vocabulary size ({vectorizer_vocab_size}) does not match model input size ({log_reg_input_size}). "
        "Ensure the vectorizer and models are from the same training pipeline."
    )


AttributeError: 'MultinomialNB' object has no attribute 'coef_'

In [179]:
import joblib

# Save the vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [25]:
import joblib
joblib.dump(log_reg_model, 'log_reg_model.pkl')
joblib.dump(gb_model, 'gb_model.pkl')
joblib.dump(rf_model, 'rf_model.pkl')
joblib.dump(nb_model, 'nb_model.pkl')

['nb_model.pkl']

In [26]:
pip install flask joblib newspaper3k

Note: you may need to restart the kernel to use updated packages.


In [4]:
from newspaper import Article

url = "https://www.bbc.com/news/articles/c6230646435o"
article = Article(url)
article.download()
article.parse()
print(article.text) 

Twenty years on: 'My boat was metres from the shore when the tsunami hit'

BBC Just as we pulled out from the harbour, our boat lurched and suddenly the jetty next to where we had boarded collapsed into the sea

Boxing Day, 2004. When the earthquake struck at 06:30 (01:00 GMT), I was on a ferry, headed towards Havelock – an island in the Indian archipelago of Andaman and Nicobar. Known for its silver sand and clear blue waters, the Radhanagar beach there had recently been crowned "Asia's Best Beach" by Time magazine. My best friend from college and her family had lived in Port Blair, the capital of the archipelago, for a decade and a half, but this was my first visit to the islands, where I had arrived on Christmas Eve. We had planned to spend three days in Havelock and in the morning we packed snacks and sandwiches, gathered excited children and headed out to catch the ferry from Phoenix Bay jetty in Port Blair. Not wanting to miss out on anything, I was standing on the front deck, lo

In [7]:
print(f"Extracted text in Jupyter: {article_text}")

Extracted text in Jupyter: Twenty years on: 'My boat was metres from the shore when the tsunami hit'

BBC Just as we pulled out from the harbour, our boat lurched and suddenly the jetty next to where we had boarded collapsed into the sea

Boxing Day, 2004. When the earthquake struck at 06:30 (01:00 GMT), I was on a ferry, headed towards Havelock – an island in the Indian archipelago of Andaman and Nicobar. Known for its silver sand and clear blue waters, the Radhanagar beach there had recently been crowned "Asia's Best Beach" by Time magazine. My best friend from college and her family had lived in Port Blair, the capital of the archipelago, for a decade and a half, but this was my first visit to the islands, where I had arrived on Christmas Eve. We had planned to spend three days in Havelock and in the morning we packed snacks and sandwiches, gathered excited children and headed out to catch the ferry from Phoenix Bay jetty in Port Blair. Not wanting to miss out on anything, I was sta

In [6]:
print(f"Vectorized text shape in Jupyter: {new_text_vectorized.shape}")

Vectorized text shape in Jupyter: (1, 2)


In [12]:
print(f"Model probabilities in Jupyter: {final_predictions}")

NameError: name 'final_predictions' is not defined

In [62]:
print(type(log_reg_model))  # This will tell you what type of model is loaded

<class 'sklearn.naive_bayes.MultinomialNB'>
